# Pokemon Data Analysis
---
Introduction to Data Analysis in python using `pandas`, `numpy`, and `matplotlib`. 

All data drawn from Kaggle's "Complete Pokemon Dataset" for Generations 1 - 7 and a web scraping of Serebii.net for Gen 8.
The sites for the original datasets are listed below: 
- [Generations 1 - 7](https://www.kaggle.com/rounakbanik/pokemon/version/1)
- [Generation 8](https://github.com/yaylinda/serebii-parser)  
  
![Pokeball](https://image.businessinsider.com/5dcee8473afd37158f6c8ab9?width=1100&format=jpeg&auto=webp)

## What Questions Should We Ask?
---
I think are large part of data analysis is simply learning to ask the right questions. Questions like "how many Pokemon are there?" are easy to answer, but do little to actually provide insight on minutia of the dataset. Instead, if focusing on quantity, we might ask "how many new Pokemon were created in each generation?". If the numbers from generation to generation seem largely similiar, we might infer that the Pokemon company sets arbitrary numeric goals for the number of new pokemon that must be introduced with every sequential release. If this does happen to be the case, we could ask further questions like "What is the distribution of typing in each release? Is it similiar? Could this effect the balance of power in the game?" or, if not, could the distribution of typing per generation be based on the in-game environment? Does generation 7, which is set in an archipelago, contain higher number of water and ground type Pokemon than other generations? However, say the generational numbers vary widely, perhaps we could ask questions about the Pokemon company itself, such as "For generations where fewer new pokemon were released, was the company's budget or timeline any different than generations with larger releases?".  
  
These are the kinds of questions that grant us further insight into the nature of the games and the company behind their development. Howewer, before we can run, we must learn to walk, so, in the interest of learning basic navigation and analysis with the libraries involved, let's start with something more readily attainable.  
  
## Basic Questions
--- 
- What is the strongest Pokemon in terms of overall stats?
- What is the most effective move typing?
    - Which is the least effective?
- Which Pokemon has the highest IV for:
    - Attack
    - Special Attack 
    - Defense
    - Special Defense
    - Speed
    - HP
- Which typing has the highest average IV's for:
    - Attack
    - Special Attack
    - Defense
    - Special Defense
    - Speed
    - HP
- Which typing has the most Pokemon over 600 IV?
- Does typing have any bearing on IV's?
    - Do fire types have stronger than average special attack?
    - Do dragon types have stronger than average attack?
    - Do rock, ground, steel types have stronger than average defense?
- What single typing has the least amount of weaknesses?
- What single typing has the highest number of weaknesses?
- What about dual typings? (Not all dual typings are currently in the game)
- Are there correlations between IV's and certain non-IV characteristics?
    - Weight and defense/hp?
    - Weight and overall total stats? (Do legendaries weigh more?)
    - Total stats and experience gain?
    - Total stats and steps-to-hatch?
  
## What Data Do We Need?
--- 
- Basic information:
    - Number 
    - Name 
- Statistically Relevant Information:
    - Typing 
    - Stats:
        - Attack
        - Special Attack 
        - Defense
        - Special Defense
        - Speed
        - HP
        - Total IV's
    - Weight
    - Experience gain
    - Steps to hatch 


## Installing Dependencies
--- 
First, we need to make sure that we have access to the libraries that we need. There are a couple of ways that we could do this, one of which involves conda, but I like `pip`. `Pip`, or the Python Installer Package, solves this pretty easily in a couple lines.

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas # the library used for handling the dataset
!{sys.executable} -m pip install matplotlib # the library used to vizualize our data
!{sys.executable} -m pip install xlrd # dependency we need to read from excel files 

## Importing The Proper Libraries
---
For this analysis, I'm going to be using `pandas` to interact with the data. Below, we will import the proper libraries and use the preferred shorthand notation for them. "`pd`" is common for pandas, while "`np`" and "`plt`" are common substitions for numpy and matplotlib's plotting functionality. `Regex` or "`re`" is also imported for some more complex sorting, but more on that later.

In [2]:
# import pandas and numpy for data analysis
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import re 

## Reading Data Into The Notebook
Pandas has a handy set of functions called read_{file extension} that allows you to import your data into a workable format called a dataframe. Both of my files are .CSV, but there are plenty of other supported extensions, including excel and standard .txt files.

In [3]:
pokemon = pd.read_excel("pokedex.xlsx")
pokemon.head()

,Pokedex Number,Name,Type 1,Type 2,HP,Attack,Defense,Special Attack,Special Defense,Speed,...,Against Ground,Against Ice,Against Normal,Against Poison,Against Psychic,Against Rock,Against Steel,Against Water,Steps to Hatch,Experience Growth
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,...,1.0,2.0,1.0,1.0,2,1.0,1.0,0.5,5120.0,1059860.0
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,...,1.0,2.0,1.0,1.0,2,1.0,1.0,0.5,5120.0,1059860.0
2,3,Venusaur,Grass,Poison,80,100,123,122,120,80,...,1.0,2.0,1.0,1.0,2,1.0,1.0,0.5,5120.0,1059860.0
3,4,Charmander,Fire,NaN,39,52,43,60,50,65,...,2.0,0.5,1.0,1.0,1,2.0,0.5,2.0,5120.0,1059860.0
4,5,Charmeleon,Fire,NaN,58,64,58,80,65,80,...,2.0,0.5,1.0,1.0,1,2.0,0.5,2.0,5120.0,1059860.0


In [4]:
# Creates a df containing only pokemon that are at least partially grass type 
grass = pokemon.loc[(pokemon["Type 1"] == "Grass") | (pokemon["Type 2"] == "Grass")]
grass

,Pokedex Number,Name,Type 1,Type 2,HP,Attack,Defense,Special Attack,Special Defense,Speed,...,Against Ground,Against Ice,Against Normal,Against Poison,Against Psychic,Against Rock,Against Steel,Against Water,Steps to Hatch,Experience Growth
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,...,1.0,2.0,1.0,1.0,2,1.0,1.0,0.50,5120.0,1059860.0
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,...,1.0,2.0,1.0,1.0,2,1.0,1.0,0.50,5120.0,1059860.0
2,3,Venusaur,Grass,Poison,80,100,123,122,120,80,...,1.0,2.0,1.0,1.0,2,1.0,1.0,0.50,5120.0,1059860.0
49,43,Oddish,Grass,Poison,45,50,55,75,65,30,...,1.0,2.0,1.0,1.0,2,1.0,1.0,0.50,5120.0,1059860.0
50,44,Gloom,Grass,Poison,60,65,70,85,75,40,...,1.0,2.0,1.0,1.0,2,1.0,1.0,0.50,5120.0,1059860.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,829,Gossifleur,Grass,NaN,40,40,60,40,60,10,...,0.5,2.0,1.0,2.0,1,1.0,1.0,0.50,NaN,NaN
860,830,Eldegoss,Grass,NaN,60,50,90,80,120,60,...,0.5,2.0,1.0,2.0,1,1.0,1.0,0.50,NaN,NaN
870,840,Applin,Grass,Dragon,40,40,80,40,40,20,...,0.5,4.0,1.0,2.0,1,1.0,1.0,0.25,NaN,NaN
871,841,Flapple,Grass,Dragon,70,110,80,95,60,70,...,0.5,4.0,1.0,2.0,1,1.0,1.0,0.25,NaN,NaN


In [7]:
grass.describe()

,Pokedex Number,HP,Attack,Defense,Special Attack,Special Defense,Speed,Generation,Is Legendary,Weight (kg),...,Against Grass,Against Ground,Against Ice,Against Normal,Against Poison,Against Rock,Against Steel,Against Water,Steps to Hatch,Experience Growth
count,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,...,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,97.000000,9.700000e+01
mean,447.009434,65.820755,74.632075,72.103774,71.707547,70.292453,57.179245,4.311321,0.047170,37.070755,...,0.450472,0.547170,2.132075,0.919811,1.820755,1.099057,1.056604,0.497642,5990.927835,1.060234e+06
std,250.779907,19.102130,29.058340,24.214096,26.097242,21.581724,26.825999,2.144154,0.213009,61.471458,...,0.177348,0.246676,0.691106,0.250112,0.700981,0.374292,0.311072,0.106320,4773.424715,1.264950e+05
min,1.000000,30.000000,27.000000,30.000000,24.000000,30.000000,10.000000,1.000000,0.000000,0.100000,...,0.250000,0.000000,1.000000,0.000000,0.000000,0.500000,0.500000,0.250000,3840.000000,6.000000e+05
25%,253.250000,50.000000,53.000000,51.250000,50.750000,55.000000,36.500000,3.000000,0.000000,5.000000,...,0.250000,0.500000,2.000000,1.000000,2.000000,1.000000,1.000000,0.500000,5120.000000,1.000000e+06
50%,462.500000,65.000000,70.000000,70.000000,65.000000,70.000000,54.500000,4.000000,0.000000,14.750000,...,0.500000,0.500000,2.000000,1.000000,2.000000,1.000000,1.000000,0.500000,5120.000000,1.059860e+06
75%,667.000000,75.000000,100.000000,85.000000,90.000000,84.250000,70.000000,6.000000,0.000000,39.150000,...,0.500000,0.500000,2.000000,1.000000,2.000000,1.000000,1.000000,0.500000,5120.000000,1.059860e+06
max,842.000000,123.000000,181.000000,131.000000,145.000000,129.000000,145.000000,8.000000,1.000000,415.600000,...,1.000000,1.000000,4.000000,1.000000,4.000000,2.000000,2.000000,1.000000,30720.000000,1.640000e+06


In [22]:
grass.sort_values(["Speed"], ascending= False)

,Pokedex Number,Name,Type 1,Type 2,HP,Attack,Defense,Special Attack,Special Defense,Speed,...,Against Ground,Against Ice,Against Normal,Against Poison,Against Psychic,Against Rock,Against Steel,Against Water,Steps to Hatch,Experience Growth
278,254,Sceptile,Grass,NaN,70,110,75,145,85,145,...,0.5,2.0,1.0,2.0,1,1.0,1.0,0.50,5120.0,1059860.0
518,492,Shaymin,Grass,Grass,100,103,75,120,75,127,...,0.5,2.0,1.0,2.0,1,1.0,1.0,0.50,30720.0,1059860.0
573,547,Whimsicott,Grass,Fairy,60,67,85,77,75,116,...,0.5,2.0,1.0,4.0,1,1.0,2.0,0.50,5120.0,1000000.0
523,497,Serperior,Grass,NaN,75,75,95,75,95,113,...,0.5,2.0,1.0,2.0,1,1.0,1.0,0.50,5120.0,1059860.0
212,189,Jumpluff,Grass,Flying,75,55,70,55,95,110,...,0.0,4.0,1.0,2.0,1,2.0,1.0,0.50,5120.0,1059860.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,840,Applin,Grass,Dragon,40,40,80,40,40,20,...,0.5,4.0,1.0,2.0,1,1.0,1.0,0.25,NaN,NaN
785,755,Morelull,Grass,Fairy,40,35,55,65,75,15,...,0.5,2.0,1.0,4.0,1,1.0,2.0,0.50,5120.0,1000000.0
619,590,Foongus,Grass,Poison,69,55,45,55,55,15,...,1.0,2.0,1.0,1.0,2,1.0,1.0,0.50,5120.0,1000000.0
626,597,Ferroseed,Grass,Steel,44,50,91,24,86,10,...,1.0,1.0,0.5,0.0,0.5,0.5,0.5,0.50,5120.0,1000000.0


## Grass Analysis
- Number of Pokemon with at least partial Grass typing:  
    106
- Averages:
    - HP: 65.8
    - Attack: 74.6
    - Defense: 72.1
    - Special Attack: 71.7
    - Special Defense: 70.3
    - Speed: 57.2
- Min/Maxing:
    - HP:
        - Max: Gogoat (123)
        - Min: Sunkern (30)
    - Attack:
        - Max: Kartana (181)
        - Min: Cottonee (27)
    - Defense:
        - Max: Kartana / Ferrothorn (131)
        - Min: Sunkern (30)
    - Special Attack:
        - Max: Sceptile (145)
        - Min: Ferroseed (24)
    - Special Defense:
        - Max: Virizion (129)
        - Min: Seedot / Sunkern / Bellsprout (30)
    - Speed:
        - Max: Sceptile (145)
        - Min: Gossifleur / Ferroseed (10)


In [5]:
fire = pokemon.loc[(pokemon["Type 1"] == "Fire") | (pokemon["Type 2"] == "Fire")]
fire

,Pokedex Number,Name,Type 1,Type 2,HP,Attack,Defense,Special Attack,Special Defense,Speed,...,Against Ground,Against Ice,Against Normal,Against Poison,Against Psychic,Against Rock,Against Steel,Against Water,Steps to Hatch,Experience Growth
3,4,Charmander,Fire,NaN,39,52,43,60,50,65,...,2.0,0.5,1.0,1.0,1,2.0,0.5,2.0,5120.0,1059860.0
4,5,Charmeleon,Fire,NaN,58,64,58,80,65,80,...,2.0,0.5,1.0,1.0,1,2.0,0.5,2.0,5120.0,1059860.0
5,6,Charizard,Fire,Flying,78,104,78,159,115,100,...,0.0,1.0,1.0,1.0,1,4.0,0.5,2.0,5120.0,1059860.0
41,37,Vulpix,Fire,NaN,38,41,40,50,65,65,...,2.0,0.5,1.0,1.0,1,2.0,0.5,2.0,5120.0,1000000.0
43,37,Ninetales,Fire,NaN,73,76,75,81,100,100,...,2.0,0.5,1.0,1.0,1,2.0,0.5,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,815,Cinderace,Fire,NaN,80,116,75,65,75,119,...,2.0,0.5,1.0,1.0,1,2.0,0.5,2.0,NaN,NaN
868,838,Carkol,Rock,Fire,80,60,90,60,70,50,...,4.0,0.5,0.5,0.5,1,2.0,1.0,4.0,NaN,NaN
869,839,Coalossal,Rock,Fire,110,80,120,80,90,30,...,4.0,0.5,0.5,0.5,1,2.0,1.0,4.0,NaN,NaN
880,850,Sizzlipede,Fire,Bug,50,65,45,50,50,45,...,1.0,0.5,1.0,1.0,1,4.0,0.5,2.0,NaN,NaN


In [29]:
fire.describe()

,Pokedex Number,HP,Attack,Defense,Special Attack,Special Defense,Speed,Generation,Is Legendary,Weight (kg),...,Against Grass,Against Ground,Against Ice,Against Normal,Against Poison,Against Rock,Against Steel,Against Water,Steps to Hatch,Experience Growth
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,...,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,62.000000,6.200000e+01
mean,451.513889,69.000000,80.486111,68.444444,88.125000,71.930556,73.722222,4.333333,0.097222,62.077778,...,0.489583,1.944444,0.562500,0.902778,0.902778,2.152778,0.513889,2.097222,7411.612903,1.081384e+06
std,267.770134,19.384908,26.936019,25.126098,30.806501,22.854917,26.095287,2.331656,0.298339,82.948444,...,0.184989,0.853970,0.181688,0.273718,0.216224,0.816377,0.110129,0.560677,6840.743463,9.137832e+04
min,4.000000,38.000000,30.000000,30.000000,15.000000,40.000000,20.000000,1.000000,0.000000,1.000000,...,0.250000,0.000000,0.250000,0.000000,0.000000,1.000000,0.250000,1.000000,2560.000000,1.000000e+06
25%,225.750000,55.000000,60.000000,49.500000,65.000000,53.500000,58.750000,2.000000,0.000000,13.000000,...,0.500000,2.000000,0.500000,1.000000,1.000000,2.000000,0.500000,2.000000,5120.000000,1.000000e+06
50%,498.500000,65.000000,79.000000,61.500000,80.500000,67.500000,72.500000,5.000000,0.000000,29.400000,...,0.500000,2.000000,0.500000,1.000000,1.000000,2.000000,0.500000,2.000000,5120.000000,1.059860e+06
75%,664.000000,80.000000,98.500000,81.250000,106.000000,86.250000,94.250000,6.000000,0.000000,78.375000,...,0.500000,2.000000,0.500000,1.000000,1.000000,2.000000,0.500000,2.000000,5120.000000,1.059860e+06
max,851.000000,115.000000,160.000000,140.000000,159.000000,154.000000,126.000000,8.000000,1.000000,430.000000,...,1.000000,4.000000,1.000000,1.000000,1.000000,4.000000,1.000000,4.000000,30720.000000,1.250000e+06


In [31]:
fire.sort_values("HP", ascending= False)

,Pokedex Number,Name,Type 1,Type 2,HP,Attack,Defense,Special Attack,Special Defense,Speed,...,Against Ground,Against Ice,Against Normal,Against Poison,Against Psychic,Against Rock,Against Steel,Against Water,Steps to Hatch,Experience Growth
268,244,Entei,Fire,NaN,115,115,85,90,75,100,...,2.0,0.5,1.0,1.0,1,2.0,0.5,2.0,20480.0,1250000.0
869,839,Coalossal,Rock,Fire,110,80,120,80,90,30,...,4.0,0.5,0.5,0.5,1,2.0,1.0,4.0,NaN,NaN
526,500,Emboar,Fire,Fighting,110,123,65,100,65,65,...,2.0,0.5,1.0,1.0,2,1.0,0.5,2.0,5120.0,1059860.0
274,250,Ho-Oh,Fire,Flying,106,130,90,110,154,90,...,0.0,1.0,1.0,1.0,1,4.0,0.5,2.0,30720.0,1250000.0
582,555,Darmanitan,Fire,Fire,105,30,105,140,105,55,...,2.0,0.5,1.0,1.0,1,2.0,0.5,2.0,5120.0,1059860.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,653,Fennekin,Fire,NaN,40,45,40,62,60,60,...,2.0,0.5,1.0,1.0,1,2.0,0.5,2.0,5120.0,1059860.0
241,218,Slugma,Fire,NaN,40,40,40,70,40,20,...,2.0,0.5,1.0,1.0,1,2.0,0.5,2.0,5120.0,1000000.0
178,155,Cyndaquil,Fire,NaN,39,52,43,60,50,65,...,2.0,0.5,1.0,1.0,1,2.0,0.5,2.0,5120.0,1059860.0
3,4,Charmander,Fire,NaN,39,52,43,60,50,65,...,2.0,0.5,1.0,1.0,1,2.0,0.5,2.0,5120.0,1059860.0


## Fire Analysis

## Basic Navigation  
---
However, notice that these datasets are quite large, especially gens 1 - 7, with over 800 rows and 41 columns. As such, not all of it is shown in the output cell. Fortunately, `pandas` has a couple helpful methods that we can use to view the parts we want to see.

First up, `.head()`. By default, this will display the first 5 rows of the dataframe, however, you can change this by adding a numerical argument for the number of rows you wish to view.  

Now, if you're thinking, "hey, that's useful, but is there another optional argument to reverse it?" The answer would technically be no, but, there is another method. This one is aptly named `.tail()`. `.tail()` functions exactly like `.head()` does, execpt it shows the *last* 5 rows in a dataframe. Again though, you can change this with an optional numeric argument.

We can use `.columns` to display the headers of our dataframe

Now, let's use these methods to reorganize our data. Currently, the dataframe headers are run alphabetically. While this might be useful in some cases, it isn't in ours. So, let's move things around to resemble a more traditional pokedex.